In [5]:
session = Session()

In [1]:
import schedule
import threading
from logger import logger_19
from db import DBOperation, Session, TddVideoRecord, TddVideoLog, TddVideoRecordAbnormalChange
import time
from util import get_ts_s, ts_s_to_str
from pybiliapi import BiliApi
import math
from common import get_valid, test_archive_rank_by_partion, add_video_record_via_stat_api, InvalidObjCodeError, \
    update_video, TddCommonError, test_video_view, add_video, AlreadyExistError
from collections import defaultdict

In [68]:
def get_need_insert_aid_list(time_label, is_tid_30, session):
    if time_label == '04:00':
        # return total
        return DBOperation.query_freq_update_video_aids(0, is_tid_30, session)

    # add 1 hour aids
    aid_list = DBOperation.query_freq_update_video_aids(2, is_tid_30, session)

    if time_label in ['00:00', '04:00', '08:00', '12:00', '16:00', '20:00']:
        # add 4 hour aids
        aid_list += DBOperation.query_freq_update_video_aids(1, is_tid_30, session)

    return aid_list

In [69]:
time_label = '18:00'

In [70]:
if True:
    task_label = ts_s_to_str(get_ts_s())[:11] + time_label
    logger_19.info('Now start hourly video task %s..' % task_label)

    bapi = BiliApi()
    session = Session()

[2020-03-19 18:23:44,890][logger_19][INFO]: Now start hourly video task 2020-03-19 18:00..


In [71]:
if True:
    logger_19.info('01: make c30 new video records from awesome api')

    c30_new_video_record_list = []

    # get page total
    obj = bapi.get_archive_rank_by_partion(30, 1, 50)
    page_total = math.ceil(obj['data']['page']['count'] / 50)
    logger_19.info('%d page(s) found' % page_total)

    page_num = 1
    while page_num <= page_total:
        # get obj via awesome api
        obj = get_valid(bapi.get_archive_rank_by_partion, (30, page_num, 50), test_archive_rank_by_partion)
        if obj is None:
            logger_19.warning('Page num %d fail! Cannot get valid obj.' % page_num)
            page_num += 1
            continue

        try:
            added = get_ts_s()
            for arch in obj['data']['archives']:
                # make new video record
                new_video_record = TddVideoRecord()
                new_video_record.aid = arch['aid']
                new_video_record.added = added
                new_video_record.view = -1 if arch['stat']['view'] == '--' else arch['stat']['view']
                new_video_record.danmaku = arch['stat']['danmaku']
                new_video_record.reply = arch['stat']['reply']
                new_video_record.favorite = arch['stat']['favorite']
                new_video_record.coin = arch['stat']['coin']
                new_video_record.share = arch['stat']['share']
                new_video_record.like = arch['stat']['like']

                c30_new_video_record_list.append(new_video_record)
            if page_num % 100 == 0:
                logger_19.info('Awesome api fetch %d / %d done' % (page_num, page_total))
        except Exception as e:
            logger_19.error('Awesome api fetch %d / %d error, Exception caught. Detail: %s' % (page_num, page_total, e))
        finally:
            page_num += 1
    logger_19.info('Awesome api fetch %d / %d done' % (page_num - 1, page_total))

    logger_19.info('01 done! c30_new_video_record_list count: %d' % len(c30_new_video_record_list))

[2020-03-19 18:23:46,381][logger_19][INFO]: 01: make c30 new video records from awesome api
[2020-03-19 18:23:46,696][logger_19][INFO]: 4453 page(s) found
[2020-03-19 18:24:00,460][logger_19][INFO]: Awesome api fetch 100 / 4453 done
[2020-03-19 18:24:10,377][logger_19][INFO]: Awesome api fetch 200 / 4453 done
[2020-03-19 18:24:19,952][logger_19][INFO]: Awesome api fetch 300 / 4453 done
[2020-03-19 18:24:28,335][logger_19][INFO]: Awesome api fetch 400 / 4453 done
[2020-03-19 18:24:38,506][logger_19][INFO]: Awesome api fetch 500 / 4453 done
[2020-03-19 18:24:50,054][logger_19][INFO]: Awesome api fetch 600 / 4453 done
[2020-03-19 18:25:09,213][logger_19][INFO]: Awesome api fetch 700 / 4453 done
[2020-03-19 18:25:20,472][logger_19][INFO]: Awesome api fetch 800 / 4453 done
[2020-03-19 18:25:41,748][logger_19][INFO]: Awesome api fetch 900 / 4453 done
[2020-03-19 18:25:59,097][logger_19][INFO]: Awesome api fetch 1000 / 4453 done
[2020-03-19 18:26:17,890][logger_19][INFO]: Awesome api fetch 11

KeyboardInterrupt: 

In [58]:
len(c30_new_video_record_list)
c30_new_video_record_list[:10]

[<TddVideoRecord(aid=97372907,view=0)>,
 <TddVideoRecord(aid=97378634,view=3)>,
 <TddVideoRecord(aid=97295443,view=0)>,
 <TddVideoRecord(aid=97376561,view=2)>,
 <TddVideoRecord(aid=97376557,view=4)>,
 <TddVideoRecord(aid=97370537,view=6)>,
 <TddVideoRecord(aid=97371922,view=4)>,
 <TddVideoRecord(aid=97367936,view=2)>,
 <TddVideoRecord(aid=97368778,view=1)>,
 <TddVideoRecord(aid=97115417,view=5277)>]

In [59]:
if True:
    logger_19.info('02: make c0 new video records and insert them')

    # get need insert c0 aids
    need_insert_c0_aid_list = get_need_insert_aid_list(time_label, is_tid_30=False, session=session)
    logger_19.info('got %d need insert c0 aids' % len(need_insert_c0_aid_list))

    c0_fail_aids = []
    c0_success_aids = []
    c0_visited = 0
    c0_new_video_record_list = []

    for aid in need_insert_c0_aid_list:
        # add video record
        try:
            new_video_record = add_video_record_via_stat_api(aid, bapi, session)
            c0_new_video_record_list.append(new_video_record)
        except InvalidObjCodeError as e:
            # update video code
            try:
                tdd_video_logs = update_video(aid, bapi, session)
            except TddCommonError as e:
                logger_19.warning('Fail to update video aid %d, Exception caught. Detail: %s' % (aid, e))
            except Exception as e:
                logger_19.error('Fail to update video aid %d, Exception caught. Detail: %s' % (aid, e))
            else:
                for log in tdd_video_logs:
                    logger_19.info('Update video aid %d, attr: %s, oldval: %s, newval: %s'
                                   % (log.aid, log.attr, log.oldval, log.newval))
            c0_fail_aids.append(aid)
        except TddCommonError as e:
            logger_19.warning('Fail to update video aid %d, Exception caught. Detail: %s', (aid, e))
            c0_fail_aids.append(aid)
        else:
            c0_success_aids.append(aid)
            logger_19.info('Add new record %s' % new_video_record)

        c0_visited += 1
        if c0_visited % 10 == 0:
            logger_19.info('c0 aid add %d / %d done' % (c0_visited, len(need_insert_c0_aid_list)))
        time.sleep(0.2)  # api duration banned

    logger_19.info('02 done! c0_total_aids count: %d, c0_success_aids count: %d, c0_fail_aids count: %d' % (
        len(need_insert_c0_aid_list), len(c0_success_aids), len(c0_fail_aids)))
    logger_19.info('c0_fail_aids: %r' % c0_fail_aids)

[2020-03-19 18:02:37,701][logger_19][INFO]: 02: make c0 new video records and insert them
[2020-03-19 18:02:37,951][logger_19][INFO]: got 7 need insert c0 aids
[2020-03-19 18:02:38,476][logger_19][INFO]: Add new record <TddVideoRecord(aid=92357413,view=92208)>
[2020-03-19 18:02:38,818][logger_19][INFO]: Add new record <TddVideoRecord(aid=92645597,view=173160)>
[2020-03-19 18:02:39,147][logger_19][INFO]: Add new record <TddVideoRecord(aid=94485762,view=35005)>
[2020-03-19 18:02:39,448][logger_19][INFO]: Add new record <TddVideoRecord(aid=93242305,view=13018)>
[2020-03-19 18:02:39,793][logger_19][INFO]: Add new record <TddVideoRecord(aid=95728280,view=23)>
[2020-03-19 18:02:40,132][logger_19][INFO]: Add new record <TddVideoRecord(aid=96442362,view=1037)>
[2020-03-19 18:02:40,454][logger_19][INFO]: Add new record <TddVideoRecord(aid=96938143,view=534)>
[2020-03-19 18:02:40,658][logger_19][INFO]: 02 done! c0_total_aids count: 7, c0_success_aids count: 7, c0_fail_aids count: 0
[2020-03-19 1

In [60]:
if True:
    logger_19.info('03: insert c30 video records')

    # go insert c30 record
    need_insert_c30_aid_list = get_need_insert_aid_list(time_label, is_tid_30=True, session=session)
    logger_19.info('got %d need insert c30 aids' % len(need_insert_c30_aid_list))

    c30_success_aids = []
    c30_visited = 0
    c30_not_added_record_list = []
    need_insert_c30_aid_list_count = len(need_insert_c30_aid_list)

    for record in c30_new_video_record_list:
        if record.aid in need_insert_c30_aid_list:
            if record.aid in c30_success_aids:
                logger_19.warning('c30 aid %d already added' % record.aid)
                continue
            need_insert_c30_aid_list.remove(record.aid)
            session.add(record)  # TODO may cause error?
            c30_success_aids.append(record.aid)
            c30_visited += 1
            if c30_visited % 100 == 0:
                try:
                    session.commit()
                except Exception as e:
                    logger_19.error('Fail to add c30 aid add %d / %d, Exception caught. Detail: %s'
                                    % (c30_visited, need_insert_c30_aid_list_count, e))
                    session.rollback()
                else:
                    logger_19.info('c30 aid add %d / %d done' % (c30_visited, need_insert_c30_aid_list_count))
        else:
            c30_not_added_record_list.append(record)
    session.commit()
    logger_19.info('c30 aid add %d / %d done' % (c30_visited, need_insert_c30_aid_list_count))

    c30_left_aids = need_insert_c30_aid_list
    logger_19.info('03 done! c30_total_aids count: %d, c30_success_aids count: %d, '
                   % (need_insert_c30_aid_list_count, len(c30_success_aids)) +
                   'c30_left_aids count: %d, c30_not_added_record_list count: %d'
                   % (len(c30_left_aids), len(c30_not_added_record_list)))

[2020-03-19 18:02:42,191][logger_19][INFO]: 03: insert c30 video records
[2020-03-19 18:02:42,373][logger_19][INFO]: got 1672 need insert c30 aids
[2020-03-19 18:02:44,480][logger_19][INFO]: c30 aid add 100 / 1672 done
[2020-03-19 18:02:47,712][logger_19][INFO]: c30 aid add 200 / 1672 done
[2020-03-19 18:02:49,703][logger_19][INFO]: c30 aid add 300 / 1672 done
[2020-03-19 18:02:51,841][logger_19][INFO]: c30 aid add 400 / 1672 done
[2020-03-19 18:02:53,941][logger_19][INFO]: c30 aid add 500 / 1672 done
[2020-03-19 18:02:56,130][logger_19][INFO]: c30 aid add 600 / 1672 done
[2020-03-19 18:02:58,115][logger_19][INFO]: c30 aid add 700 / 1672 done
[2020-03-19 18:03:00,349][logger_19][INFO]: c30 aid add 800 / 1672 done
[2020-03-19 18:03:02,655][logger_19][INFO]: c30 aid add 900 / 1672 done
[2020-03-19 18:03:04,813][logger_19][INFO]: c30 aid add 1000 / 1672 done
[2020-03-19 18:03:07,199][logger_19][INFO]: c30 aid add 1100 / 1672 done
[2020-03-19 18:03:09,427][logger_19][INFO]: c30 aid add 120

In [61]:
c30_left_aids

[96040304, 96681531]

In [62]:
if True:
    logger_19.info('04: check left aids, change tid or code')

    # there aids should got record from awesome api, but now seems they dont
    # might because they are now tid != 30 or code != 0
    c30_left_unsolved_aids = []  # fail to handle
    c30_left_tid_changed_aids = []  # tid changed 30 -> new tid
    c30_left_code_changed_aids = []  # code changed 0 -> new code
    c30_left_added_aids = []  # tid = 30 and code = 0, add new video record

    logger_19.info('got %d c30 left aids' % len(c30_left_aids))
    for aid in c30_left_aids:
        time.sleep(0.2)  # api duration banned
        # get view obj
        view_obj = get_valid(bapi.get_video_view, (aid,), test_video_view)
        if view_obj is None:
            logger_19.warning('Aid %d fail! Cannot get valid view obj.' % aid)
            c30_left_unsolved_aids.append(aid)
            continue

        # record view obj request ts
        view_obj_added = get_ts_s()

        try:
            # get video code
            code = view_obj['code']

            if code == 0:
                # code==0, check tid next
                if 'tid' in view_obj['data'].keys():
                    # get video tid
                    tid = view_obj['data']['tid']
                    if tid != 30:
                        # video tid!=30 now, change tid
                        try:
                            old_video = DBOperation.query_video_via_aid(aid, session)
                            session.add(TddVideoLog(view_obj_added, aid, 'tid', old_video.tid, tid))
                            old_video.tid = tid
                            session.add(TddVideoLog(view_obj_added, aid, 'isvc', old_video.isvc, 5))
                            old_video.isvc = 5
                            session.commit()
                            logger_19.info(
                                'Update video aid = %d tid from 30 to %d then update isvc = %d.'
                                % (aid, tid, 5))
                            c30_left_tid_changed_aids.append(aid)
                        except Exception as e:
                            session.rollback()
                            logger_19.warning(
                                'Fail to update video aid = %d tid from 30 to %d then update isvc = %d. ' % (
                                    aid, tid, 5) + 'Exception caught. Detail: %s' % e)
                            c30_left_unsolved_aids.append(aid)
                    else:
                        # video tid==30, add video record
                        # logger_19.warning(
                        #     'Found aid = %d code == 0 and tid == 30! Now try add video record...' % aid)

                        # get stat first
                        stat = view_obj['data']['stat']

                        # make new tdd video record obj and assign stat info from api
                        new_video_record = TddVideoRecord()
                        new_video_record.aid = aid
                        new_video_record.added = view_obj_added
                        new_video_record.view = -1 if stat['view'] == '--' else stat['view']
                        new_video_record.danmaku = stat['danmaku']
                        new_video_record.reply = stat['reply']
                        new_video_record.favorite = stat['favorite']
                        new_video_record.coin = stat['coin']
                        new_video_record.share = stat['share']
                        new_video_record.like = stat['like']

                        # add to db
                        DBOperation.add(new_video_record, session)
                        logger_19.info('Add record %s.' % new_video_record)
                        c30_left_added_aids.append(aid)
                else:
                    logger_19.error('View obj %s got code == 0 but no tid field! Need further check!' % view_obj)
                    c30_left_unsolved_aids.append(aid)
            else:
                # code!=0, change code
                try:
                    old_video = DBOperation.query_video_via_aid(aid, session)
                    session.add(TddVideoLog(view_obj_added, aid, 'code', old_video.code, code))
                    old_video.code = code
                    session.commit()
                    logger_19.info('Update video aid = %d code from 0 to %d.' % (aid, code))
                    c30_left_code_changed_aids.append(aid)
                except Exception as e:
                    session.rollback()
                    logger_19.warning('Fail to update video aid = %d code from 0 to %d.' % (aid, code) +
                                      'Exception caught. Detail: %s' % e)
                    c30_left_unsolved_aids.append(aid)
        except Exception as e:
            logger_19.error(
                'Exception caught when process view obj of left aid %d. Detail: %s' % (aid, e))
            c30_left_unsolved_aids.append(aid)

    logger_19.info('04 done. c30_left_aids count: %d, c30_left_unsolved_aids count: %d, '
                   % (len(c30_left_aids), len(c30_left_unsolved_aids)) +
                   'c30_left_tid_changed_aids count: %d, c30_left_code_changed_aids count: %d, '
                   % (len(c30_left_tid_changed_aids), len(c30_left_code_changed_aids)) +
                   'c30_left_added_aids: %d' % len(c30_left_added_aids))

[2020-03-19 18:03:27,723][logger_19][INFO]: 04: check left aids, change tid or code
[2020-03-19 18:03:27,724][logger_19][INFO]: got 2 c30 left aids
[2020-03-19 18:03:28,152][logger_19][INFO]: Add record <TddVideoRecord(aid=96040304,view=80)>.
[2020-03-19 18:03:28,569][logger_19][INFO]: Update video aid = 96681531 code from 0 to 62002.
[2020-03-19 18:03:28,569][logger_19][INFO]: 04 done. c30_left_aids count: 2, c30_left_unsolved_aids count: 0, c30_left_tid_changed_aids count: 0, c30_left_code_changed_aids count: 1, c30_left_added_aids: 1


In [22]:
# TODO 04 check forward video
# http://api.bilibili.com/x/web-interface/view?aid=96040304
# "state": -4,
# "forward": 96040259,

In [63]:
if True:
    logger_19.info('05: check c30 not added aid record list and add them')

    c30_not_added_add_video_aids = []
    if time_label == '04:00':
        logger_19.info('got %d c30 not added records' % len(c30_not_added_record_list))

        # check not added record list
        for record in c30_not_added_record_list:
            time.sleep(0.2)  # api duration banned
            aid = record.aid
            # add video
            try:
                new_video = add_video(aid, bapi, session)
            except AlreadyExistError:
                # video already exist, which is absolutely common
                pass
            except TddCommonError as e:
                logger_19.warning('Fail to add video aid %d. Exception caught. Detail: %s' % (aid, e))
                continue
            else:
                logger_19.info('Add new video %s' % new_video)
                c30_not_added_add_video_aids.append(aid)

            # add video record
            DBOperation.add(record, session)

        logger_19.info('05 done! c30_not_added_add_video_aids count: %d' % len(c30_not_added_add_video_aids))
    else:
        logger_19.info('05 done! time label is not 04:00, no need to process them')

[2020-03-19 18:03:36,861][logger_19][INFO]: 05: check c30 not added aid record list and add them
[2020-03-19 18:03:36,862][logger_19][INFO]: 05 done! time label is not 04:00, no need to process them


In [64]:
if True:
    # save to file
    new_video_record_list = c30_new_video_record_list + c0_new_video_record_list
    filename = 'data/%s.csv' % task_label
    with open(filename, 'w') as f:
        f.write('aid,added,view,danmaku,reply,favorite,coin,share,like\n')
        for record in new_video_record_list:
            f.write('%d,%d,%d,%d,%d,%d,%d,%d,%d\n'
                    % (record.aid, record.added, record.view, record.danmaku,
                       record.reply, record.favorite, record.coin, record.share, record.like))
    index_filename = 'data/index.txt'
    with open(index_filename, 'a') as f:
        f.write('%s\n' % filename)

    logger_19.info('06 done! %d record(s) stored in file %s' % (len(new_video_record_list), filename))

[2020-03-19 18:04:32,030][logger_19][INFO]: 06 done! 222649 record(s) stored in file data/2020-03-19 17:00.csv


In [65]:
if True:
    logger_19.info('07: load records from history files')

    # load records from history files
    history_filename_list = []
    with open(index_filename, 'r') as f:
        lines = f.readlines()
        for line in lines[-25:]:
            history_filename_list.append(line.rstrip('\n'))
    logger_19.info('Will load records from file list %r' % history_filename_list)

    history_record_dict = defaultdict(list)
    history_record_count = 0
    for filename in history_filename_list:
        with open(filename, 'r') as f:
            lines = f.readlines()
            for line in lines[1:]:
                line_list = line.rstrip('\n').split(',')
                video_record = TddVideoRecord()
                video_record.aid = int(line_list[0])
                video_record.added = int(line_list[1])
                video_record.view = int(line_list[2])
                video_record.danmaku = int(line_list[3])
                video_record.reply = int(line_list[4])
                video_record.favorite = int(line_list[5])
                video_record.coin = int(line_list[6])
                video_record.share = int(line_list[7])
                video_record.like = int(line_list[8])
                video_record_list = history_record_dict[video_record.aid]
                video_record_list.append(video_record)
                history_record_count += 1
        logger_19.info('Finish load records from file %s' % filename)

    logger_19.info('07 done! loaded %d history records from %d files'
                   % (history_record_count, len(history_filename_list)))

[2020-03-19 18:04:36,234][logger_19][INFO]: 07: load records from history files
[2020-03-19 18:04:36,244][logger_19][INFO]: Will load records from file list ['data/2020-03-19 15:00.csv', 'data/2020-03-19 17:00.csv']
[2020-03-19 18:04:41,925][logger_19][INFO]: Finish load records from file data/2020-03-19 15:00.csv
[2020-03-19 18:04:47,465][logger_19][INFO]: Finish load records from file data/2020-03-19 17:00.csv
[2020-03-19 18:04:47,466][logger_19][INFO]: 07 done! loaded 445287 history records from 2 files


In [66]:
history_record_dict

defaultdict(list,
            {97336214: [<TddVideoRecord(aid=97336214,view=0)>,
              <TddVideoRecord(aid=97336214,view=8)>],
             97334996: [<TddVideoRecord(aid=97334996,view=14)>,
              <TddVideoRecord(aid=97334996,view=84)>],
             97330804: [<TddVideoRecord(aid=97330804,view=1)>,
              <TddVideoRecord(aid=97330804,view=2)>],
             97329095: [<TddVideoRecord(aid=97329095,view=1)>,
              <TddVideoRecord(aid=97329095,view=3)>],
             97533120: [<TddVideoRecord(aid=97533120,view=1)>,
              <TddVideoRecord(aid=97533120,view=9)>],
             97326636: [<TddVideoRecord(aid=97326636,view=2)>,
              <TddVideoRecord(aid=97326636,view=8)>],
             97333666: [<TddVideoRecord(aid=97333666,view=7)>,
              <TddVideoRecord(aid=97333666,view=155)>],
             97327119: [<TddVideoRecord(aid=97327119,view=1)>,
              <TddVideoRecord(aid=97327119,view=1)>],
             97331484: [<TddVideoRecord(ai

In [ ]:
if True:
    logger_19.info('08: check params of history video records')

    # check record
    for record in new_video_record_list:
        video_record_list = history_record_dict[record.aid]
        video_record_list.sort(key=lambda r: r.added)
        # TODO check params
        

In [46]:
d = {
    2: 200,
    3: 20,
    1: 100,
    4: 30
}
l = [(k, v) for (k, v) in d.items()]
l

[(2, 200), (3, 20), (1, 100), (4, 30)]

In [47]:
sl = sorted(l)
sl

[(1, 100), (2, 200), (3, 20), (4, 30)]

[(1, 100), (2, 200), (3, 20), (4, 30)]

In [2]:
from db import Session, TddVideoRecord

if True:
    session = Session()
    record = TddVideoRecord()
    record.added = 0
    record.aid = 456930
    record.view = 1
    record.danmaku = 2
    record.reply = 3
    record.favorite = 4
    record.coin = 5
    record.share = 6
    record.like = 7
    session.add(record)
    session.commit()

In [5]:
if True:
    print(record.id)

43668680


In [7]:
session.add(record)
session.commit()

In [9]:
print(record.id)

43668680


In [10]:
if True:
    record = TddVideoRecord()
    record.added = 0
    record.aid = 456930
    record.view = 1
    record.danmaku = 2
    record.reply = 3
    record.favorite = 4
    record.coin = 5
    record.share = 6
    record.like = 7
    print(record.id)

None


In [ ]:
    record = TddVideoRecord()
    record.added = 0
    record.aid = 456930
    record.view = 1
    record.danmaku = 2
    record.reply = 3
    record.favorite = 4
    record.coin = 5
    record.share = 6
    record.like = 7
    session.add(record)
    session.commit()

In [37]:
if True:
    # load records from history files
    index_filename = 'data/index.txt'
    history_filename_list = []
    with open(index_filename, 'r') as f:
        lines = f.readlines()
        for line in lines[-25:-10]:
            history_filename_list.append(line.rstrip('\n'))
    logger_19.info('Will load records from file list %r' % history_filename_list)

[2020-03-20 20:59:16,376][logger_19][INFO]: Will load records from file list ['data/2020-03-19 19:00.csv', 'data/2020-03-19 20:00.csv', 'data/2020-03-19 21:00.csv', 'data/2020-03-19 22:00.csv', 'data/2020-03-19 23:00.csv', 'data/2020-03-20 00:00.csv', 'data/2020-03-20 01:00.csv', 'data/2020-03-20 02:00.csv', 'data/2020-03-20 03:00.csv', 'data/2020-03-20 04:00.csv', 'data/2020-03-20 05:00.csv', 'data/2020-03-20 06:00.csv', 'data/2020-03-20 07:00.csv', 'data/2020-03-20 08:00.csv', 'data/2020-03-20 09:00.csv']


In [17]:
from collections import namedtuple

In [18]:
VideoRecord = namedtuple("VideoRecord", ['aid', 'added', 'view', 'danmaku', 'reply', 'favorite', 'coin', 'share', 'like'])

In [40]:
if True:
    history_record_dict = defaultdict(list)
    history_record_count = 0
    for filename in history_filename_list:
        try:
            with open(filename, 'r') as f:
                lines = f.readlines()
                for line in lines[1:]:
                    try:
                        line_list = line.rstrip('\n').split(',')
                        video_record = VideoRecord(
                            int(line_list[0]),
                            int(line_list[1]),
                            int(line_list[2]),
                            int(line_list[3]),
                            int(line_list[4]),
                            int(line_list[5]),
                            int(line_list[6]),
                            int(line_list[7]),
                            int(line_list[8]),
                        )
                        #video_record.aid = int(line_list[0])
                        #video_record.added = int(line_list[1])
                        #video_record.view = int(line_list[2])
                        #video_record.danmaku = int(line_list[3])
                        #video_record.reply = int(line_list[4])
                        #video_record.favorite = int(line_list[5])
                        #video_record.coin = int(line_list[6])
                        #video_record.share = int(line_list[7])
                        #video_record.like = int(line_list[8])
                        video_record_list = history_record_dict[video_record.aid]
                        video_record_list.append(video_record)
                    except Exception as e:
                        logger_19.warning('Fail to make video record from line: %s. Exception caught. Detail: %s'
                                          % (line, e))
                    finally:
                        history_record_count += 1
            logger_19.info('Finish load records from file %s' % filename)
        except Exception as e:
            logger_19.warning('Fail to read load records from file %s. Exception caught. Detail: %s' % (filename, e))

[2020-03-20 20:59:56,991][logger_19][INFO]: Finish load records from file data/2020-03-19 19:00.csv
[2020-03-20 20:59:57,923][logger_19][INFO]: Finish load records from file data/2020-03-19 20:00.csv
[2020-03-20 20:59:58,841][logger_19][INFO]: Finish load records from file data/2020-03-19 21:00.csv
[2020-03-20 20:59:59,845][logger_19][INFO]: Finish load records from file data/2020-03-19 22:00.csv
[2020-03-20 21:00:00,885][logger_19][INFO]: Finish load records from file data/2020-03-19 23:00.csv
[2020-03-20 21:00:01,971][logger_19][INFO]: Finish load records from file data/2020-03-20 00:00.csv
[2020-03-20 21:00:02,814][logger_19][INFO]: Finish load records from file data/2020-03-20 01:00.csv
[2020-03-20 21:00:03,917][logger_19][INFO]: Finish load records from file data/2020-03-20 02:00.csv
[2020-03-20 21:00:04,764][logger_19][INFO]: Finish load records from file data/2020-03-20 03:00.csv
[2020-03-20 21:00:05,527][logger_19][INFO]: Finish load records from file data/2020-03-20 04:00.csv


In [41]:
if True:
    # get video pubdate
    session = Session()
    video_pubdate_list = DBOperation.query_video_pubdate_all(session)
    video_pubdate_dict = dict()
    for (aid, pubdate) in video_pubdate_list:
        video_pubdate_dict[aid] = pubdate
    logger_19.info('Finish make video pubdate dict with %d aids.' % len(video_pubdate_dict))

[2020-03-20 21:00:50,404][logger_19][INFO]: Finish make video pubdate dict with 238994 aids.


In [54]:
if True:
    check_visited_count = 0
    check_total_count = len(history_record_dict)
    for aid in history_record_dict.keys():
        video_record_list = history_record_dict[aid]
        if len(video_record_list) <= 2:  # at least require 3 record
            continue

        video_record_list.sort(key=lambda r: r.added)

        timespan_now = video_record_list[-1].added - video_record_list[-2].added
        if timespan_now == 0:
            logger_19.warning('%d got timespan_now = 0, continue')
            continue
        speed_now_dict = dict()
        speed_now_dict['view'] = (video_record_list[-1].view - video_record_list[-2].view) / timespan_now * 3600
        speed_now_dict['danmaku'] = (video_record_list[-1].danmaku - video_record_list[-2].danmaku) / timespan_now * 3600
        speed_now_dict['reply'] = (video_record_list[-1].reply - video_record_list[-2].reply) / timespan_now * 3600
        speed_now_dict['favorite'] = (video_record_list[-1].favorite - video_record_list[-2].favorite) / timespan_now * 3600
        speed_now_dict['coin'] = (video_record_list[-1].coin - video_record_list[-2].coin) / timespan_now * 3600
        speed_now_dict['share'] = (video_record_list[-1].share - video_record_list[-2].share) / timespan_now * 3600
        speed_now_dict['like'] = (video_record_list[-1].like - video_record_list[-2].like) / timespan_now * 3600

        timespan_last = video_record_list[-2].added - video_record_list[-3].added
        if timespan_last == 0:
            logger_19.warning('%d got timespan_last = 0, continue')
            continue
        speed_last_dict = dict()
        speed_last_dict['view'] = (video_record_list[-2].view - video_record_list[-3].view) / timespan_last * 3600
        speed_last_dict['danmaku'] = (video_record_list[-2].danmaku - video_record_list[-3].danmaku) / timespan_last * 3600
        speed_last_dict['reply'] = (video_record_list[-2].reply - video_record_list[-3].reply) / timespan_last * 3600
        speed_last_dict['favorite'] = (video_record_list[-2].favorite - video_record_list[-3].favorite) / timespan_last * 3600
        speed_last_dict['coin'] = (video_record_list[-2].coin - video_record_list[-3].coin) / timespan_last * 3600
        speed_last_dict['share'] = (video_record_list[-2].share - video_record_list[-3].share) / timespan_last * 3600
        speed_last_dict['like'] = (video_record_list[-2].like - video_record_list[-3].like) / timespan_last * 3600

        speed_now_incr_rate_dict = dict()
        speed_now_incr_rate_dict['view'] = (speed_now_dict['view'] - speed_last_dict['view']) / (speed_last_dict['view'] + 0.01)
        speed_now_incr_rate_dict['danmaku'] = (speed_now_dict['danmaku'] - speed_last_dict['danmaku']) / (speed_last_dict['danmaku'] + 0.01)
        speed_now_incr_rate_dict['reply'] = (speed_now_dict['reply'] - speed_last_dict['reply']) / (speed_last_dict['reply'] + 0.01)
        speed_now_incr_rate_dict['favorite'] = (speed_now_dict['favorite'] - speed_last_dict['favorite']) / (speed_last_dict['favorite'] + 0.01)
        speed_now_incr_rate_dict['coin'] = (speed_now_dict['coin'] - speed_last_dict['coin']) / (speed_last_dict['coin'] + 0.01)
        speed_now_incr_rate_dict['share'] = (speed_now_dict['share'] - speed_last_dict['share']) / (speed_last_dict['share'] + 0.01)
        speed_now_incr_rate_dict['like'] = (speed_now_dict['like'] - speed_last_dict['like']) / (speed_last_dict['like'] + 0.01)

        period_range = video_record_list[-1].added - video_record_list[0].added
        if period_range == 0:
            logger_19.warning('%d got period_range = 0, continue')
            continue

        speed_period_dict = dict()
        speed_period_dict['view'] = (video_record_list[-1].view - video_record_list[0].view) / period_range * 3600
        speed_period_dict['danmaku'] = (video_record_list[-1].danmaku - video_record_list[0].danmaku) / period_range * 3600
        speed_period_dict['reply'] = (video_record_list[-1].reply - video_record_list[0].reply) / period_range * 3600
        speed_period_dict['favorite'] = (video_record_list[-1].favorite - video_record_list[0].favorite) / period_range * 3600
        speed_period_dict['coin'] = (video_record_list[-1].coin - video_record_list[0].coin) / period_range * 3600
        speed_period_dict['share'] = (video_record_list[-1].share - video_record_list[0].share) / period_range * 3600
        speed_period_dict['like'] = (video_record_list[-1].like - video_record_list[0].like) / period_range * 3600

        overall_range = video_record_list[-1].added - video_pubdate_dict[aid] if aid in video_pubdate_dict.keys() and video_pubdate_dict[aid] else video_record_list[-1].added
        if overall_range == 0:
            logger_19.warning('%d got overall_range = 0, continue')
            continue
        
        speed_overall_dict = dict()
        speed_overall_dict['view'] = video_record_list[-1].view / overall_range * 3600
        speed_overall_dict['danmaku'] = video_record_list[-1].danmaku / overall_range * 3600
        speed_overall_dict['reply'] = video_record_list[-1].reply / overall_range * 3600
        speed_overall_dict['favorite'] = video_record_list[-1].favorite / overall_range * 3600
        speed_overall_dict['coin'] = video_record_list[-1].coin / overall_range * 3600
        speed_overall_dict['share'] = video_record_list[-1].share / overall_range * 3600
        speed_overall_dict['like'] = video_record_list[-1].like / overall_range * 3600
        
        
#         print(video_record_list[-3:])
#         print('speed_now', speed_now_dict)
#         print('speed_last', speed_last_dict)
#         print('speed_now_incr_rate', speed_now_incr_rate_dict)
#         print('speed_period', speed_period_dict)
#         if check_visited_count == 5:
#             break

        has_abnormal_change = False
        new_change_list = []

        # check unexpected drop
        for (key, value) in speed_now_dict.items():
            if value < -10:
                new_change = TddVideoRecordAbnormalChange()
                new_change.added = video_record_list[-1].added
                new_change.aid = aid
                new_change.attr = key
                new_change.speed_now = speed_now_dict[key]
                new_change.speed_last = speed_last_dict[key]
                new_change.speed_now_incr_rate = speed_now_incr_rate_dict[key]
                new_change.period_range = period_range
                new_change.speed_overall = speed_overall_dict[key]
                new_change.description = 'unexpected drop detected, speed now of %s is %f, < -10' % (key, value)
                logger_19.info('%d change: %s' % (aid, new_change.description))
                has_abnormal_change = True
                new_change_list.append(new_change)

        # check unexpected increase speed
        for (key, value) in speed_now_incr_rate_dict.items():
            if value > 2 and speed_now_dict[key] > 50:
                new_change = TddVideoRecordAbnormalChange()
                new_change.added = video_record_list[-1].added
                new_change.aid = aid
                new_change.attr = key
                new_change.speed_now = speed_now_dict[key]
                new_change.speed_last = speed_last_dict[key]
                new_change.speed_now_incr_rate = speed_now_incr_rate_dict[key]
                new_change.period_range = period_range
                new_change.speed_overall = speed_overall_dict[key]
                new_change.description = 'unexpected increase speed detected, speed now of {0} is {1}%, > 200%'.format(
                    key, value * 100)
                logger_19.info('%d change: %s' % (aid, new_change.description))
                has_abnormal_change = True
                new_change_list.append(new_change)

#         if has_abnormal_change and record.id is None:
#             DBOperation.add(record, session)
#             logger_19.info('Add video record %s' % record)

        # TODO change freq

#         try:
#             for new_change in new_change_list:
#                 new_change.record_id = record.id
#                 session.add(new_change)
#             session.commit()
#         except Exception as e:
#             logger_19.error('Fail to add new change list with aid %d. Exception caught. Detail: %s' % (record.aid, e))

        check_visited_count += 1
        if check_visited_count % 10000 == 0:
            logger_19.info('check %d / %d done' % (check_visited_count, check_total_count))
        
    logger_19.info('check %d / %d done' % (check_visited_count, check_total_count))

[2020-03-20 21:15:03,133][logger_19][INFO]: 97129985 change: unexpected increase speed detected, speed now of danmaku is 1539.2303848075962%, > 200%
[2020-03-20 21:15:03,306][logger_19][INFO]: check 10000 / 223013 done
[2020-03-20 21:15:03,471][logger_19][INFO]: 76977749 change: unexpected drop detected, speed now of view is -23.000000, < -10
[2020-03-20 21:15:03,474][logger_19][INFO]: check 20000 / 223013 done
[2020-03-20 21:15:03,641][logger_19][INFO]: check 30000 / 223013 done
[2020-03-20 21:15:03,739][logger_19][WARNING]: %d got timespan_last = 0, continue
[2020-03-20 21:15:03,815][logger_19][INFO]: check 40000 / 223013 done
[2020-03-20 21:15:03,976][logger_19][INFO]: check 50000 / 223013 done
[2020-03-20 21:15:04,147][logger_19][INFO]: check 60000 / 223013 done
[2020-03-20 21:15:04,158][logger_19][INFO]: 36570568 change: unexpected increase speed detected, speed now of reply is 514.5959765685301%, > 200%
[2020-03-20 21:15:04,317][logger_19][INFO]: check 70000 / 223013 done
[2020-0

In [11]:
import re
s = 'From madoka_homura@bunnyxt.com Sat Jun 5'
ss = re.findall('\S+?@\S+', s)
ss

['madoka_homura@bunnyxt.com']

In [12]:
ss = re.findall('m\S+?@\S+', s)
ss

['madoka_homura@bunnyxt.com']

In [13]:
ss = re.findall('m\S+@\S+', s)
ss

['madoka_homura@bunnyxt.com']

In [15]:
s = 'From madoka@homura@bunnyxt.com Sat Jun 5'
ss = re.findall('\S+?@', s)
ss

['madoka@', 'homura@']

In [16]:
s = 'From madoka@homura@bunnyxt.com Sat Jun 5'
ss = re.findall('\S+@', s)
ss

['madoka@homura@']

In [22]:
from db import Session
session = Session()
r = session.execute('select distinct(record_id) from tdd_video_record_abnormal_change where id <= 54 && id <= 120;')
r

In [23]:
aa = []
for _ in r:
    aa.append(_[0])
aa

[43695804,
 43696527,
 43704698,
 43936586,
 43940157,
 43940158,
 43940159,
 43937319,
 43937370,
 43937378,
 43937461,
 43937989,
 43939028,
 43939045,
 43939087,
 43939144,
 43940160,
 43939204,
 43939829,
 43939840,
 43940973,
 43942706,
 43942721,
 43942724,
 43943933,
 43945325,
 43945345,
 43945347,
 43951742,
 43952322,
 43952611,
 43953013,
 43954473,
 43954775,
 43957230,
 43955441,
 43957231,
 43957234,
 43957540,
 43957548,
 43960030,
 43960031,
 43960033]

In [24]:
for a in aa:
    sql = 'delete from tdd_video where id = ' + str(a)
    session.execute(sql)
session.commit()

In [25]:
session.close()

In [8]:
import pymysql
db = pymysql.connect(
    host='114.67.86.202',
    user='td',
    passwd='TDDaily_2019',
    port=3306,
    db='tdd',
    charset='GBK')
cursor = db.cursor()

In [9]:
sql = 'select * from tdd_video_record_abnormal_change where id <= 145;'
cursor.execute(sql)
results = cursor.fetchall()

In [15]:
c2 = db.cursor()
for row in results:
    _id = row[0]
    sql = 'select * from tdd_video_record where id = ' + str(row[10])
    c2.execute(sql)
    rr = c2.fetchall()
    for r in rr:
        sql = 'update tdd_video_record_abnormal_change set this_added = {0}, this_view = {1}, this_danmaku = {2}, this_reply = {3}, this_favorite = {4}, this_coin = {5}, this_share = {6}, this_like = {7} where id = {8}'.format(
        r[1], r[3], r[4], r[5], r[6], r[7], r[8], r[9], _id
        )
        cursor.execute(sql)
        db.commit()

In [14]:
db.commit()

In [16]:
c2.close()

In [28]:
for a in aa:
    sql = 'delete from tdd_video where id = ' + str(a)
    cursor.execute(sql)
db.commit()

In [17]:
cursor.close()
db.close()

ZeroDivisionError: division by zero

In [52]:
float('inf') * -1111

-inf

In [58]:
b = 200
c = 0
a = (b - c) / c if c != 0 else float('inf') * (b - c)
a

inf

In [59]:
a > 2

True

In [61]:
from db import TddVideoRecordAbnormalChange

In [68]:
new_change = TddVideoRecordAbnormalChange()
new_change.added = 1
new_change.aid = 2
new_change.attr = 'test'
new_change.speed_now = 1
new_change.speed_last = 0
new_change.speed_now_incr_rate = 'value(1e500)'
new_change.period_range = 1
new_change.speed_period = 1
new_change.speed_overall = 1
new_change.description = 'test'
new_change.record_id = 1

In [64]:
session = Session()

In [69]:
session.add(new_change)

In [70]:
session.commit()

DataError: (pymysql.err.DataError) (1265, "Data truncated for column 'speed_now_incr_rate' at row 1")
[SQL: INSERT INTO tdd_video_record_abnormal_change (added, aid, attr, speed_now, speed_last, speed_now_incr_rate, period_range, speed_period, speed_overall, record_id, description, comment) VALUES (%(added)s, %(aid)s, %(attr)s, %(speed_now)s, %(speed_last)s, %(speed_now_incr_rate)s, %(period_range)s, %(speed_period)s, %(speed_overall)s, %(record_id)s, %(description)s, %(comment)s)]
[parameters: {'added': 1, 'aid': 2, 'attr': 'test', 'speed_now': 1, 'speed_last': 0, 'speed_now_incr_rate': 'value(1e500)', 'period_range': 1, 'speed_period': 1, 'speed_overall': 1, 'record_id': 1, 'description': 'test', 'comment': ''}]
(Background on this error at: http://sqlalche.me/e/9h9h)

In [71]:
session.rollback()

In [2]:
import schedule
import threading
from logger import logger_19
from db import DBOperation, Session, TddVideoRecord, TddVideoLog, TddVideoRecordAbnormalChange
import time
from util import get_ts_s, ts_s_to_str
from pybiliapi import BiliApi
import math
from common import get_valid, test_archive_rank_by_partion, add_video_record_via_stat_api, InvalidObjCodeError, \
    update_video, TddCommonError, test_video_view, add_video, AlreadyExistError
from collections import defaultdict, namedtuple
import gc

In [3]:
# 07
if True:
    logger_19.info('07: load records from history files')

    # load records from history files
    history_filename_list = []
    index_filename = 'data/index.txt'
    with open(index_filename, 'r') as f:
        lines = f.readlines()
        for line in lines[-13:]:
            history_filename_list.append(line.rstrip('\n'))
    logger_19.info('Will load records from file list %r' % history_filename_list)

    VideoRecord = namedtuple("VideoRecord",
                             ['aid', 'added', 'view', 'danmaku', 'reply', 'favorite', 'coin', 'share', 'like'])
    history_record_dict = defaultdict(list)
    history_record_count = 0
    for filename in history_filename_list:
        try:
            with open(filename, 'r') as f:
                lines = f.readlines()
                for line in lines[1:]:
                    try:
                        line_list = line.rstrip('\n').split(',')
                        video_record = VideoRecord(
                            int(line_list[0]),
                            int(line_list[1]),
                            int(line_list[2]),
                            int(line_list[3]),
                            int(line_list[4]),
                            int(line_list[5]),
                            int(line_list[6]),
                            int(line_list[7]),
                            int(line_list[8]),
                        )
                        video_record_list = history_record_dict[video_record.aid]
                        video_record_list.append(video_record)
                    except Exception as e:
                        logger_19.warning('Fail to make video record from line: %s. Exception caught. Detail: %s'
                                          % (line, e))
                    finally:
                        history_record_count += 1
            logger_19.info('Finish load records from file %s' % filename)
        except Exception as e:
            logger_19.warning('Fail to read load records from file %s. Exception caught. Detail: %s' % (filename, e))

    logger_19.info('07 done! loaded %d history records from %d files'
                   % (history_record_count, len(history_filename_list)))

[2020-03-22 14:53:06,730][logger_19][INFO]: 07: load records from history files
[2020-03-22 14:53:06,732][logger_19][INFO]: Will load records from file list ['data/2020-03-22 01:00.csv', 'data/2020-03-22 02:00.csv', 'data/2020-03-22 03:00.csv', 'data/2020-03-22 05:00.csv', 'data/2020-03-22 06:00.csv', 'data/2020-03-22 07:00.csv', 'data/2020-03-22 08:00.csv', 'data/2020-03-22 09:00.csv', 'data/2020-03-22 10:00.csv', 'data/2020-03-22 11:00.csv', 'data/2020-03-22 12:00.csv', 'data/2020-03-22 13:00.csv', 'data/2020-03-22 14:00.csv']
[2020-03-22 14:53:07,850][logger_19][INFO]: Finish load records from file data/2020-03-22 01:00.csv
[2020-03-22 14:53:08,683][logger_19][INFO]: Finish load records from file data/2020-03-22 02:00.csv
[2020-03-22 14:53:09,777][logger_19][INFO]: Finish load records from file data/2020-03-22 03:00.csv
[2020-03-22 14:53:10,554][logger_19][INFO]: Finish load records from file data/2020-03-22 05:00.csv
[2020-03-22 14:53:11,600][logger_19][INFO]: Finish load records f

In [6]:
# 08
if True:
    logger_19.info('08: check params of history video records')

    # get video pubdate
    video_pubdate_list = DBOperation.query_video_pubdate_all(session)
    video_pubdate_dict = dict()
    for (aid, pubdate) in video_pubdate_list:
        video_pubdate_dict[aid] = pubdate
    logger_19.info('Finish make video pubdate dict with %d aids.' % len(video_pubdate_dict))

[2020-03-22 14:53:39,181][logger_19][INFO]: 08: check params of history video records
[2020-03-22 14:54:13,727][logger_19][INFO]: Finish make video pubdate dict with 239498 aids.


In [7]:
# 08
if True:

    # check record
#     check_total_count = len(new_video_record_list)
    check_total_count = len(history_record_dict)
    check_visited_count = 0
#     for record in new_video_record_list:
    for aid in history_record_dict.keys():
        video_record_list = history_record_dict[aid]
        if len(video_record_list) <= 2:  # at least require 3 record
            continue
        
        record = video_record_list[-1]

        video_record_list.sort(key=lambda r: r.added)

        # remove all zero situation
        if record.view == 0 and record.danmaku == 0 and record.reply == 0 and record.favorite == 0 and \
                record.coin == 0 and record.share == 0 and record.like == 0:
            logger_19.warning('%d got all params of record = 0, maybe API bug, continue', record.aid)
            continue

        # remove abnormal all zero VideoRecord
        abnormal_all_zero_index_list = []
        for i in range(len(video_record_list)):
            video_record = video_record_list[i]
            if video_record.view == 0 and video_record.danmaku == 0 and video_record.reply == 0 and \
                    video_record.favorite == 0 and video_record.coin == 0 and video_record.share == 0 and \
                    video_record.like == 0:
                if i == 0:
                    abnormal_all_zero_index_list.append(i)  # start from all zero, remove it
                else:
                    video_record_last = video_record_list[i - 1]
                    if video_record_last.view == 0 and video_record_last.danmaku == 0 and video_record_last.reply == 0 and \
                            video_record_last.favorite == 0 and video_record_last.coin == 0 and video_record_last.share == 0 and \
                            video_record_last.like == 0:
                        pass
                    else:
                        abnormal_all_zero_index_list.append(i)  # from not all zero to zero, remove it
        for i in reversed(abnormal_all_zero_index_list):
            logger_19.warning('%d found abnormal all zero video record at %d, delete it'
                              % (record.aid, video_record_list[i].added))
            del video_record_list[i]

        if len(video_record_list) <= 2:  # at least require 3 record
            continue

        timespan_now = video_record_list[-1].added - video_record_list[-2].added
        if timespan_now == 0:
            logger_19.warning('%d got timespan_now = 0, continue', record.aid)
            continue
        speed_now_dict = dict()
        speed_now_dict['view'] = (video_record_list[-1].view - video_record_list[-2].view) / timespan_now * 3600
        speed_now_dict['danmaku'] = (video_record_list[-1].danmaku - video_record_list[-2].danmaku) / timespan_now * 3600
        speed_now_dict['reply'] = (video_record_list[-1].reply - video_record_list[-2].reply) / timespan_now * 3600
        speed_now_dict['favorite'] = (video_record_list[-1].favorite - video_record_list[-2].favorite) / timespan_now * 3600
        speed_now_dict['coin'] = (video_record_list[-1].coin - video_record_list[-2].coin) / timespan_now * 3600
        speed_now_dict['share'] = (video_record_list[-1].share - video_record_list[-2].share) / timespan_now * 3600
        speed_now_dict['like'] = (video_record_list[-1].like - video_record_list[-2].like) / timespan_now * 3600

        timespan_last = video_record_list[-2].added - video_record_list[-3].added
        if timespan_last == 0:
            logger_19.warning('%d got timespan_last = 0, continue', record.aid)
            continue
        speed_last_dict = dict()
        speed_last_dict['view'] = (video_record_list[-2].view - video_record_list[-3].view) / timespan_last * 3600
        speed_last_dict['danmaku'] = (video_record_list[-2].danmaku - video_record_list[-3].danmaku) / timespan_last * 3600
        speed_last_dict['reply'] = (video_record_list[-2].reply - video_record_list[-3].reply) / timespan_last * 3600
        speed_last_dict['favorite'] = (video_record_list[-2].favorite - video_record_list[-3].favorite) / timespan_last * 3600
        speed_last_dict['coin'] = (video_record_list[-2].coin - video_record_list[-3].coin) / timespan_last * 3600
        speed_last_dict['share'] = (video_record_list[-2].share - video_record_list[-3].share) / timespan_last * 3600
        speed_last_dict['like'] = (video_record_list[-2].like - video_record_list[-3].like) / timespan_last * 3600

        speed_now_incr_rate_dict = dict()
        # speed_now_incr_rate_dict['view'] = (speed_now_dict['view'] - speed_last_dict['view']) / (speed_last_dict['view'] + 0.01)
        # speed_now_incr_rate_dict['danmaku'] = (speed_now_dict['danmaku'] - speed_last_dict['danmaku']) / (speed_last_dict['danmaku'] + 0.01)
        # speed_now_incr_rate_dict['reply'] = (speed_now_dict['reply'] - speed_last_dict['reply']) / (speed_last_dict['reply'] + 0.01)
        # speed_now_incr_rate_dict['favorite'] = (speed_now_dict['favorite'] - speed_last_dict['favorite']) / (speed_last_dict['favorite'] + 0.01)
        # speed_now_incr_rate_dict['coin'] = (speed_now_dict['coin'] - speed_last_dict['coin']) / (speed_last_dict['coin'] + 0.01)
        # speed_now_incr_rate_dict['share'] = (speed_now_dict['share'] - speed_last_dict['share']) / (speed_last_dict['share'] + 0.01)
        # speed_now_incr_rate_dict['like'] = (speed_now_dict['like'] - speed_last_dict['like']) / (speed_last_dict['like'] + 0.01)
        speed_now_incr_rate_dict['view'] = (speed_now_dict['view'] - speed_last_dict['view']) \
            / speed_last_dict['view'] if speed_last_dict['view'] != 0 else \
            99999999 * (speed_now_dict['view'] - speed_last_dict['view'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['view'] - speed_last_dict['view'])
        speed_now_incr_rate_dict['danmaku'] = (speed_now_dict['danmaku'] - speed_last_dict['danmaku']) \
            / speed_last_dict['danmaku'] if speed_last_dict['danmaku'] != 0 else \
            99999999 * (speed_now_dict['danmaku'] - speed_last_dict['danmaku'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['danmaku'] - speed_last_dict['danmaku'])
        speed_now_incr_rate_dict['reply'] = (speed_now_dict['reply'] - speed_last_dict['reply']) \
            / speed_last_dict['reply'] if speed_last_dict['reply'] != 0 else \
            99999999 * (speed_now_dict['reply'] - speed_last_dict['reply'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['reply'] - speed_last_dict['reply'])
        speed_now_incr_rate_dict['favorite'] = (speed_now_dict['favorite'] - speed_last_dict['favorite']) \
            / speed_last_dict['favorite'] if speed_last_dict['favorite'] != 0 else \
            99999999 * (speed_now_dict['favorite'] - speed_last_dict['favorite'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['favorite'] - speed_last_dict['favorite'])
        speed_now_incr_rate_dict['coin'] = (speed_now_dict['coin'] - speed_last_dict['coin']) \
            / speed_last_dict['coin'] if speed_last_dict['coin'] != 0 else \
            99999999 * (speed_now_dict['coin'] - speed_last_dict['coin'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['coin'] - speed_last_dict['coin'])
        speed_now_incr_rate_dict['share'] = (speed_now_dict['share'] - speed_last_dict['share']) \
            / speed_last_dict['share'] if speed_last_dict['share'] != 0 else \
            99999999 * (speed_now_dict['share'] - speed_last_dict['share'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['share'] - speed_last_dict['share'])
        speed_now_incr_rate_dict['like'] = (speed_now_dict['like'] - speed_last_dict['like']) \
            / speed_last_dict['like'] if speed_last_dict['like'] != 0 else \
            99999999 * (speed_now_dict['like'] - speed_last_dict['like'])  # magic number to represent infinity
            # float('inf') * (speed_now_dict['like'] - speed_last_dict['like'])

        period_range = video_record_list[-1].added - video_record_list[0].added
        if period_range == 0:
            logger_19.warning('%d got period_range = 0, continue', record.aid)
            continue

        speed_period_dict = dict()
        speed_period_dict['view'] = (video_record_list[-1].view - video_record_list[0].view) / period_range * 3600
        speed_period_dict['danmaku'] = (video_record_list[-1].danmaku - video_record_list[0].danmaku) / period_range * 3600
        speed_period_dict['reply'] = (video_record_list[-1].reply - video_record_list[0].reply) / period_range * 3600
        speed_period_dict['favorite'] = (video_record_list[-1].favorite - video_record_list[0].favorite) / period_range * 3600
        speed_period_dict['coin'] = (video_record_list[-1].coin - video_record_list[0].coin) / period_range * 3600
        speed_period_dict['share'] = (video_record_list[-1].share - video_record_list[0].share) / period_range * 3600
        speed_period_dict['like'] = (video_record_list[-1].like - video_record_list[0].like) / period_range * 3600

        overall_range = video_record_list[-1].added
        if record.aid in video_pubdate_dict.keys() and video_pubdate_dict[record.aid]:
            overall_range -= video_pubdate_dict[record.aid]
        if overall_range == 0:
            logger_19.warning('%d got overall_range = 0, continue', record.aid)
            continue

        speed_overall_dict = dict()
        speed_overall_dict['view'] = video_record_list[-1].view / overall_range * 3600
        speed_overall_dict['danmaku'] = video_record_list[-1].danmaku / overall_range * 3600
        speed_overall_dict['reply'] = video_record_list[-1].reply / overall_range * 3600
        speed_overall_dict['favorite'] = video_record_list[-1].favorite / overall_range * 3600
        speed_overall_dict['coin'] = video_record_list[-1].coin / overall_range * 3600
        speed_overall_dict['share'] = video_record_list[-1].share / overall_range * 3600
        speed_overall_dict['like'] = video_record_list[-1].like / overall_range * 3600

        has_abnormal_change = False
        new_change_list = []

        # check unexpected drop
        for (key, value) in speed_now_dict.items():
            if value < -50:
                new_change = TddVideoRecordAbnormalChange()
                new_change.added = video_record_list[-1].added
                new_change.aid = record.aid
                new_change.attr = key
                new_change.speed_now = speed_now_dict[key]
                new_change.speed_last = speed_last_dict[key]
                new_change.speed_now_incr_rate = speed_now_incr_rate_dict[key]
                new_change.period_range = period_range
                new_change.speed_period = speed_period_dict[key]
                new_change.speed_overall = speed_overall_dict[key]
                new_change.this_added = video_record_list[-1].added
                new_change.this_view = video_record_list[-1].view
                new_change.this_danmaku = video_record_list[-1].danmaku
                new_change.this_reply = video_record_list[-1].reply
                new_change.this_favorite = video_record_list[-1].favorite
                new_change.this_coin = video_record_list[-1].coin
                new_change.this_share = video_record_list[-1].share
                new_change.this_like = video_record_list[-1].like
                new_change.last_added = video_record_list[-2].added
                new_change.last_view = video_record_list[-2].view
                new_change.last_danmaku = video_record_list[-2].danmaku
                new_change.last_reply = video_record_list[-2].reply
                new_change.last_favorite = video_record_list[-2].favorite
                new_change.last_coin = video_record_list[-2].coin
                new_change.last_share = video_record_list[-2].share
                new_change.last_like = video_record_list[-2].like
                new_change.description = 'unexpected drop detected, speed now of %s is %f, < -50' % (key, value)
                logger_19.info('%d change: %s' % (record.aid, new_change.description))
                has_abnormal_change = True
                new_change_list.append(new_change)

        # check unexpected increase speed
        for (key, value) in speed_now_incr_rate_dict.items():
            if value > 2 and speed_now_dict[key] > 50:
                new_change = TddVideoRecordAbnormalChange()
                new_change.added = video_record_list[-1].added
                new_change.aid = record.aid
                new_change.attr = key
                new_change.speed_now = speed_now_dict[key]
                new_change.speed_last = speed_last_dict[key]
                new_change.speed_now_incr_rate = speed_now_incr_rate_dict[key]
                new_change.period_range = period_range
                new_change.speed_period = speed_period_dict[key]
                new_change.speed_overall = speed_overall_dict[key]
                new_change.this_added = video_record_list[-1].added
                new_change.this_view = video_record_list[-1].view
                new_change.this_danmaku = video_record_list[-1].danmaku
                new_change.this_reply = video_record_list[-1].reply
                new_change.this_favorite = video_record_list[-1].favorite
                new_change.this_coin = video_record_list[-1].coin
                new_change.this_share = video_record_list[-1].share
                new_change.this_like = video_record_list[-1].like
                new_change.last_added = video_record_list[-2].added
                new_change.last_view = video_record_list[-2].view
                new_change.last_danmaku = video_record_list[-2].danmaku
                new_change.last_reply = video_record_list[-2].reply
                new_change.last_favorite = video_record_list[-2].favorite
                new_change.last_coin = video_record_list[-2].coin
                new_change.last_share = video_record_list[-2].share
                new_change.last_like = video_record_list[-2].like
                new_change.description = 'unexpected increase speed detected, speed now of {0} is {1}%, > 200%'.format(
                    key, value * 100)
                logger_19.info('%d change: %s' % (record.aid, new_change.description))
                has_abnormal_change = True
                new_change_list.append(new_change)

#         if has_abnormal_change and record.id is None:
#             DBOperation.add(record, session)
#             logger_19.info('Add video record %s' % record)

        # TODO change freq

        try:
            for new_change in new_change_list:
#                 new_change.this_record_id = record.id
                # TODO make add last record to tdd_video_record
                session.add(new_change)
            session.commit()
        except Exception as e:
            logger_19.error('Fail to add new change list with aid %d. Exception caught. Detail: %s' % (record.aid, e))

        check_visited_count += 1
        if check_visited_count % 10000 == 0:
            logger_19.info('check %d / %d done' % (check_visited_count, check_total_count))

    logger_19.info('check %d / %d done' % (check_visited_count, check_total_count))

    logger_19.info('08 done! Finish check params of history video records')

[2020-03-22 14:55:02,657][logger_19][WARNING]: 98041434 found abnormal all zero video record at 1584810000, delete it
[2020-03-22 14:55:02,659][logger_19][WARNING]: 98028314 got all params of record = 0, maybe API bug, continue
[2020-03-22 14:55:02,664][logger_19][WARNING]: 97963669 found abnormal all zero video record at 1584810000, delete it
[2020-03-22 14:55:02,670][logger_19][WARNING]: 97870104 found abnormal all zero video record at 1584810001, delete it
[2020-03-22 14:55:02,686][logger_19][WARNING]: 97527813 found abnormal all zero video record at 1584810001, delete it
[2020-03-22 14:55:02,689][logger_19][WARNING]: 97480314 got all params of record = 0, maybe API bug, continue
[2020-03-22 14:55:02,709][logger_19][WARNING]: 97002547 got all params of record = 0, maybe API bug, continue
[2020-03-22 14:55:02,711][logger_19][WARNING]: 96987962 got all params of record = 0, maybe API bug, continue
[2020-03-22 14:55:02,740][logger_19][WARNING]: 96489097 got all params of record = 0, ma